## 感知机

### 感知机示图
![jupyter](./感知机.jpg)

## 其中 {x1....xn} 为输入值， {w1 ... wn}为各特征的权重， w0 为偏移值 可以看作为值为1的权重 

## step funcation 激活函数， 有很多种激活函数， 比如常见的sigmoid, softmax, relu 等（这里以阶跃函数为例）
## 阶跃函数: ![jupyter](./阶跃函数.png)


## 感知机输出可以由下面公式来计算：![jupyter](./感知机输出函数.png)


### 感知机模拟实现and函数

### and函数的输入， 输出对应
![jupyter](./and输入输出.png)


In [3]:
def f(x):
    return 1 if x > 0 else 0

In [4]:
### 我们令w1 = 0.5, w2 = 0.5, b = -0.8 
### 当x1, x2为0时：
w1 = 0.5
w2 = 0.5
b = -0.8
x1 = 0
x2 = 0
y = f(w1*x1 + w2*x2 +b)
y

0

In [5]:
### 当x1 = 0, x2 = 1 时：
x1 = 0
x2 = 1
y = f(w1*x1 + w2*x2 +b)
y

0

In [6]:
### 当x1 = 1, x2 = 0 时：
x1 = 1
x2 = 0
y = f(w1*x1 + w2*x2 +b)
y

0

In [7]:
### 当x1 = 1, x2 = 1 时：
x1 = 1
x2 = 1
y = f(w1*x1 + w2*x2 +b)
y

1

### 感知机模拟实现or函数
### or 的输入输出对应：
![jupyter](./or函数输入输出.png)

In [8]:
### 我们令w1 = 0.5, w2 = 0.5, b = -0.3
### 当x1, x2为0时：
w1 = 0.5
w2 = 0.5
b = -0.3
x1 = 0
x2 = 0
y = f(w1*x1 + w2*x2 +b)
y

0

In [9]:
### 当x1 = 0, x2 = 1时
x1 = 0
x2 = 1
y = f(w1*x1 + w2*x2 +b)
y

1

In [10]:
### 当x1 = 1, x2 = 0时
x1 = 1
x2 = 0
y = f(w1*x1 + w2*x2 +b)
y

1

In [11]:
### 当x1 = 1, x2 = 1时
x1 = 1
x2 = 1
y = f(w1*x1 + w2*x2 +b)
y

1

### 感知机的训练 （如何更新权重， 偏移值)

### 感知机是如何训练的？ 
![jupyter](./感知机训练1.png)
### 其中
![jupyter](./感知机训练2.png)

### 其中 η 表示学习率， 一个常数 用于控制每一步权重更新的的幅度， t为实际值， y为当前权重下的预测值 w为权重， b 为偏移量（可视为值为1的权重）
### 1。 初始化感知机的权重， 偏移值为0 
### 2。 根据当前感知机的权重， 以及输入值， 计算出当前的输出 
### 3。 根据输出值与真实值， 更新权重， 偏移量。 
### 根据需要，当述步骤循环n次 

### 感知机实现

In [17]:
import numpy as np
from functools import reduce

In [12]:
class Perceptron(object):
    ''' 初始化感知机，
    input_num： 输入参数个数
    activator: 激活函数
    '''
    def __init__(self, input_num, activator):
        self.activator = activator
        # 权重初始化为0, 偏移量为0
        self.weights = [0.0 for _ in range(input_num)]
        self.bias = 0.0

    ##返回感知机信息
    def __str__(self):
         return 'weights\t %s \nbias\t: %f\n' % (list(self.weights), self.bias)

    ''' 预测结果
    input_vec: 输出要预测的向量
    '''
    def predict(self, input_vec):
        # 把input_vec[x1,x2,x3...]和weights[w1,w2,w3,...]打包在一起
        # 变成[(x1,w1),(x2,w2),(x3,w3),...]
        # 然后利用map函数计算[x1*w1, x2*w2, x3*w3]
        # 最后利用reduce求和
        # python2 -- > python3 lambda (x, w): x * w  --> lambda x_w: x_w[0]*x_w[1]
        ret = self.activator(reduce(lambda a, b: a+b,
                                    map(lambda x_w: x_w[0]*x_w[1], zip(input_vec, self.weights))
                                    , 0.0) + self.bias)
        return ret

    ''' 训练数据
    input_vecs: 训练数据集 train_x
    labels:  标签  train_y
    iteration: 迭代次数
    rate: 学习率
    '''
    def train(self, input_vecs, labels, iteration, rate):
        for i in range(iteration):
            self._one_iteration(input_vecs, labels, rate)

    ''' 训练数据 （一次迭代，把所有训练集过一遍）
        input_vecs: 训练数据集 train_x
        labels:  标签  train_y
        rate: 学习率
        '''
    def _one_iteration(self, input_vecs, labels, rate):
        # 把输入和输出打包在一起，成为样本的列表[(input_vec, label), ...]
        # 而每个训练样本是(input_vec, label)
        print(self.__str__())
        samples = zip(input_vecs, labels)
        for (input_vec, label) in samples:
            output = self.predict(input_vec)
            # 更新权重
            self._update_weights(input_vec, output, label, rate)


    ''' 更新权重
    input_vec: 当前数据
    output: 当前数据预测值
    label:  当前数据标签
    rate: 学习率
    '''
    def _update_weights(self, input_vec, output, label, rate):
        # 把input_vec[x1,x2,x3,...]和weights[w1,w2,w3,...]打包在一起
        # 变成[(x1,w1),(x2,w2),(x3,w3),...]
        # 然后利用感知器规则更新权重
        # python2 -- > python3 lambda (x, w): x * w  --> lambda x_w: x_w[0]*x_w[1]
        delta = label - output
        self.weights = list(map(lambda x_w: x_w[1] + rate* delta * x_w[0], zip(input_vec, self.weights)))
        # 更新偏移量
        self.bias += rate*delta

### 测试感知机

In [13]:
'''
训练数据集（基于实现 and操作（逻辑与））
'''
def training_dataset():
    input_vecs = [[1,1], [0,0], [1,0], [0,1]]
    labels = [1, 0, 0, 0]
    return input_vecs, labels


In [14]:
''' 使用感知机训练数据集
'''
def train_and_perceptron():
    p = Perceptron(2, f)
    input_vecs, labels = training_dataset()
    ## 迭代10次 学习率为0。1
    p.train(input_vecs, labels, 10, 0.1)
    # 返回训练好的感知器
    return p

In [15]:
'''
定义激活函数
'''
def f(x):
    return 1 if x > 0 else 0

In [18]:
if __name__ == '__main__':
    # 训练and感知器
    train_perception = train_and_perceptron()
    # 打印训练获得的权重
    print(train_perception)
    # 测试
    print('1 and 1 = %d' % train_perception.predict([1, 1]))
    print('0 and 0 = %d' % train_perception.predict([0, 0]))
    print('1 and 0 = %d' % train_perception.predict([1, 0]))
    print('0 and 1 = %d' % train_perception.predict([0, 1]))

weights	 [0.0, 0.0] 
bias	: 0.000000

weights	 [0.0, 0.1] 
bias	: -0.100000

weights	 [0.0, 0.1] 
bias	: -0.200000

weights	 [0.1, 0.1] 
bias	: -0.200000

weights	 [0.1, 0.2] 
bias	: -0.200000

weights	 [0.1, 0.2] 
bias	: -0.200000

weights	 [0.1, 0.2] 
bias	: -0.200000

weights	 [0.1, 0.2] 
bias	: -0.200000

weights	 [0.1, 0.2] 
bias	: -0.200000

weights	 [0.1, 0.2] 
bias	: -0.200000

weights	 [0.1, 0.2] 
bias	: -0.200000

1 and 1 = 1
0 and 0 = 0
1 and 0 = 0
0 and 1 = 0


In [19]:
'''
训练数据集（基于实现 or操作（逻辑或））
'''
def training_dataset1():
    input_vecs = [[1,1], [0,0], [1,0], [0,1]]
    labels = [1, 0, 1, 1]
    return input_vecs, labels


In [20]:
''' 使用感知机训练数据集
'''
def train_and_perceptron():
    p = Perceptron(2, f)
    input_vecs, labels = training_dataset1()
    ## 迭代10次 学习率为0。1
    p.train(input_vecs, labels, 10, 0.1)
    # 返回训练好的感知器
    return p

In [21]:
if __name__ == '__main__':
    # 训练and感知器
    train_perception = train_and_perceptron()
    # 打印训练获得的权重
    print(train_perception)
    # 测试
    print('1 or 1 = %d' % train_perception.predict([1, 1]))
    print('0 or 0 = %d' % train_perception.predict([0, 0]))
    print('1 or 0 = %d' % train_perception.predict([1, 0]))
    print('0 or 1 = %d' % train_perception.predict([0, 1]))

weights	 [0.0, 0.0] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

weights	 [0.1, 0.1] 
bias	: 0.000000

1 or 1 = 1
0 or 0 = 0
1 or 0 = 1
0 or 1 = 1


### 小结：
### 感知机可以用于回归， 也可以用于分类（像and,or 就可以认为是一种二分类算法） 
### 单个感知机能实现and, or 函数， 但不能实现xor(异或）函数， 因为xor线性不可分
### 异或：
![jupyter](./异或.png)

### 要实现异或， 可使用多层感知机（也称人工神经网络ANN）